##### 감성분석 -> 머신러닝
- 데이터셋 : 전처리
- BoW 모델
    - 단어를 특성 벡터로 변환
    - tf-idf 단어 적합성 평가
    - 텍스트 데이터 정제
    - 문서를 토큰으로 나누기
- LogisticRegession

In [23]:
# 데이터셋
# http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

In [24]:
import pandas as pd
from glob import glob
file_list = glob('C:\\Users\\sally\\SKNPYWORKSPACE\\data\\movie\\train\\neg\\*txt')
pd_lists = []
for file_path in file_list[:500]:
    with open(file_path, 'r', encoding='utf-8') as f:
        data = {
            'review' : f.read(),
            'target' : 0
        }
        df = pd.DataFrame([data])
        pd_lists.append(df)

train_neg_df = pd.concat(pd_lists, ignore_index=True)
train_neg_df.head()

,review,target
0,Story of a man who has unnatural feelings for ...,0
1,Airport '77 starts as a brand new luxury 747 p...,0
2,This film lacked something I couldn't put my f...,0
3,"Sorry everyone,,, I know this is supposed to b...",0
4,When I was little my parents took me along to ...,0


In [25]:
# positive 동일하게 train_pos_df
# train_df = pd.concat([train_neg_df, train_pos_df])
# movie_data.csv로 저장

# 긍정 리뷰 불러오기
file_list = glob('C:\\Users\\sally\\SKNPYWORKSPACE\\data\\movie\\train\\pos\\*txt')
pd_lists = []
for file_path in file_list[:500]:
    with open(file_path, 'r', encoding='utf-8') as f:
        data = {
            'review': f.read(),
            'target': 1   # 긍정 레이블
        }
        df = pd.DataFrame([data])
        pd_lists.append(df)

train_pos_df = pd.concat(pd_lists, ignore_index=True)

# 부정 + 긍정 합치기
train_df = pd.concat([train_neg_df, train_pos_df], ignore_index=True)
train_df.head()

# CSV 저장
train_df.to_csv('C:\\Users\\sally\\SKNPYWORKSPACE\\data\\movie_data.csv', index=False, encoding='utf-8')

In [ ]:
# BoW(Bag of Words) 모델
# 문자를 숫자벡터
# 단어의 등장횟수를 카운트

# 전체 훈련데이터에서 모든 고유한 단어(토큰)로 어휘 사전
# 각 문서(리뷰데이터)를 사전을 기준으로 벡터화  N번째단어가 문서에서 3번나오면 벡터의 N번째값이 3이 된다.
# 문서1 : "나는 영화가 좋다"
# 문서2 : "나는 영화가 싫다"
# 사전 : {'나는':0, '영화가':1,'좋다':2,'싫다':3}
# 벡터화는 사전의 크기만큼 모든 문장의 길이를 동일하게
# 문서1벡터 : [0,1,2] - > [1,1,1,0]
# 문서2벡터 : [0,1,3] - > [1,1,0,1]

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
    "The sun is shining",
    "The weather is sweet"
])
bag = count.fit_transform(docs)

In [28]:
count.vocabulary_

{'the': 4, 'sun': 2, 'is': 0, 'shining': 1, 'weather': 5, 'sweet': 3}

In [29]:
# 문서 d에 등장한 단어 t의 횟수를 tf(t,d)
# BoW를 보완하면서 좀더 정교환 텍스트 벡터화 방식  TF-IDF(Term Frequency -Inverse Document Frequency)
# TF : 특정문서에서 자주 등장하는 단어
# IDF : 전체문서에 드물게 등장하는 단어
# 특정문서에서 자주 등장하지만 전체 문장에서 드물게 등장하는 단어에 높은 가중치를 부여 - 그 문장을 잘 대표하는 핵심 단어를 찾는다
# TF(t,d)  단어 t가 문장 d에 나타난 횟수 / 문서d의 모든 단어수
# IDF(t,D) : log( 총 문서수|D| / 단어 t를 포함한 문서의수 df(t) )  -- log 단어의 희귀성을 너무 과하게 반영하지 않도록 스케일링
# 분모에 + 1(사이킷런의 경우) :  분모가 0되는 것을 방지
# log( 1+ |D| / 1+df(f)  )

#TF-IDF(t,d,D) = TF(t,d) x IDF(t,D)

# "나는"
# TF : 리뷰에 3번 나옴 (높음)
# IDF : 전체 10,000개 리뷰중에 9000개 나옴(매우 낮음)
# tf-idf  높음 x 매우낮음 = 낮음(중요도가 낮음)

# "명작" TF : 리뷰에 2번 나옴 (높음)
# IDF : 전체 10,000개 리뷰중에 50개 나옴(매우 높음)
# tf-idf  높음 x 매우높음 = 높음(핵심단어)

In [30]:
# 데이터 정재... html tag 와 같은 불필요한 string이 보임... 특수기호 기타등등.  < - ' 
import re
def preprocessor(s):
    # 1. 영문, 공백, ., , 만 남기기
    clean = re.sub(r'[^A-Za-z\s.,]+', '', s)
    # 2. 연속된 마침표(...)를 마침표 하나로
    clean = re.sub(r'\.{2,}', '.', clean)
    # 3. 연속된 공백 정리
    clean = re.sub(r'\s+', ' ', clean).strip()
    return clean

In [31]:
df['review'] = df.review.apply(preprocessor)


In [34]:
# 문서를 토큰으로 나누기
# %pip install nltk
# %conda install -c conda-forge nltk

In [ ]:
from nltk.stem.porter import PorterStemmer

def tokenizer(text):
    return text.split()

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

# 어간추출 Stemming  단어의 접미사 -s -es -ing -ed 등등.. 를 강제로 제거해서 단어의 원형을 찾는과정
tokenizer(df.review[0][:100])

['Many',
 'of',
 'the',
 'criticisms',
 'on',
 'this',
 'thread',
 'seem',
 'to',
 'pick',
 'a',
 'comparison',
 'of',
 'this',
 'film',
 'with',
 'The',
 'Mortal',
 'Storm',
 'o']

In [37]:
tokenizer('runners like running')

['runners', 'like', 'running']

In [ ]:
# 불용어
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')    # 불용어 사전 다운로드
stops =  stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sally\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [ ]:
df.head()  # 전처리완료  정규식을 이용한..

In [ ]:
from sklearn.model_selection import train_test_split
X = df.review
y = df.target
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)

In [ ]:
# 어간추출(postStemmer) -> stopwor에 포함된 단어제거
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split() if word not in stops]

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [ ]:
tfidf = TfidfVectorizer(
    tokenizer=tokenizer_porter,
    ngram_range=(1,1) # (1,1) 유니그램(unigramm, 단일단어)만 사용
)
pipeline = Pipeline([
    ('tfidf', tfidf),
    ('clf', LogisticRegression())
])

In [ ]:
pipeline.fit(x_train,y_train)


NameError: name 'x_test' is not defined

In [ ]:
pipeline.score(x_test,y_test)


In [ ]:
# 원본데이터로드..
# 토크나이져 함수를 정의
    # 텍스트 전처리
    # 공백을 기준으로 단어단위로 분리
    # 영어는 전부 소문자로 변환
    # 어간 추출
    # 불용어 제거
# TFIDF를 정의
    # 토크나이져 매개변수 = 토크나이져 함수
    # ngram  (1,1)
# 파이프라인으로 tfidf, 머신러닝
# 파이프라이으로 학습
# 파이프라인으로 평가( classification_report)
# 과적합여부 확인

# train 폴더에 있는데이터로 학습 - 적당한 크기로
# test 폴더에 있는 문장으로 평가